# 1.2 EntropyMeasure - Randomized Measurement
## Multiple Experiments

Consider a scenario, you have multiple circuits that you want to run at once.

Call `.measure()` one by one will be inefficient,
no to mention that you also need to call `.anlyze()` for their post-processing.

Here we provide a more efficient way solve this problem,
where the true power of Qurrium as experiment manage toolkit.


### a. Import the instances


In [1]:
from qurry import EntropyMeasure

experiment_randomized = EntropyMeasure()
# It's default method. EntropyMeasure(method='randomized') also works

/home/nccu/Documents/.homejupyter/qurryplayground/qurrium_docs/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### b. Preparing quantum circuit

Prepare and add circuits to the `.wave` for later usage.


In [2]:
from qiskit import QuantumCircuit
from qurry.recipe import TrivialParamagnet, GHZ


def make_neel_circuit(n):
    qc = QuantumCircuit(n)
    for i in range(0, n, 2):
        qc.x(i)
    return qc


for i in range(2, 13, 2):
    experiment_randomized.add(TrivialParamagnet(i), f"trivial_paramagnet_{i}")
    experiment_randomized.add(GHZ(i), f"ghz_{i}")
    experiment_randomized.add(make_neel_circuit(i), f"neel_{i}")

experiment_randomized.waves

WaveContainer({
  'trivial_paramagnet_2': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x73540e5a8f20>,
  'ghz_2': <qurry.recipe.simple.cat.GHZ object at 0x7353e3981790>,
  'neel_2': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7353e39a5160>,
  'trivial_paramagnet_4': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x73545cfa8110>,
  'ghz_4': <qurry.recipe.simple.cat.GHZ object at 0x7353e3a74c20>,
  'neel_4': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7353e3916c00>,
  'trivial_paramagnet_6': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x7353e3ae97f0>,
  'ghz_6': <qurry.recipe.simple.cat.GHZ object at 0x7353e39cbb60>,
  'neel_6': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7353e3a75130>,
  'trivial_paramagnet_8': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x7353e39cbef0>,
  'ghz_8': <qurry.recipe.simple.cat.GHZ object at 0x7353e39f40e0>,
  'neel_8': <qiskit.circuit.quantumcircuit.Quantu

### c. Execute multiple experiments at once

Let's demonstrate the true power of Qurrium.


In [3]:
from qurry.qurrent.randomized_measure.arguments import EntropyMeasureRandomizedMeasureArgs

Preparing a configuration list for multiple experiments with following parameters:

```python
class EntropyMeasureRandomizedMeasureArgs(total=False):
    """Output arguments for :meth:`output`."""
    shots: int
    """Number of shots."""
    tags: Optional[tuple[str, ...]]
    """The tags to be used for the experiment."""

    wave: Optional[Union[QuantumCircuit, Hashable]]
    """The key or the circuit to execute."""
    times: int
    """The number of random unitary operator. 
    It will denote as `N_U` in the experiment name."""
    measure: Optional[Union[tuple[int, int], int, list[int]]]
    """The measure range."""
    unitary_loc: Optional[Union[tuple[int, int], int, list[int]]]
    """The range of the unitary operator."""
    unitary_loc_not_cover_measure: bool
    """Whether the range of the unitary operator is not cover the measure range."""
    random_unitary_seeds: Optional[dict[int, dict[int, int]]]
    """The seeds for all random unitary operator.
    This argument only takes input as type of `dict[int, dict[int, int]]`.
    The first key is the index for the random unitary operator.
    The second key is the index for the qubit.

    .. code-block:: python
        {
            0: {0: 1234, 1: 5678},
            1: {0: 2345, 1: 6789},
            2: {0: 3456, 1: 7890},
        }

    If you want to generate the seeds for all random unitary operator,
    you can use the function :func:`generate_random_unitary_seeds` 
    in :mod:`qurry.qurrium.utils.random_unitary`.

    .. code-block:: python
        from qurry.qurrium.utils.random_unitary import generate_random_unitary_seeds

        random_unitary_seeds = generate_random_unitary_seeds(100, 2)
    """
```


In [4]:
config_list: list[EntropyMeasureRandomizedMeasureArgs] = [
    {
        "shots": 1024,
        "wave": f"{wave_names}_{i}",
        "times": 100,
        "tags": (wave_names, f"size_{i}"),
    }
    for _ in range(10)
    for i in range(2, 13, 2)
    for wave_names in ["trivial_paramagnet", "ghz", "neel"]
]
print(len(config_list))

180


The `.multiOutput` will return an id of this `multimanager` instance,
which can be used to get the results and post-process them.

Each `multimanager` will export the experiments in a folder you can specify
by setting `save_location` parameter with default location for current directory
where Python executed.
It will create a folder with the name of the `multimanager` instance,
and inside it will create a folder for storing each experiment data.

It will do firstly in the building process, but you can skip it by setting `skip_build_write=True` to save time.
After all experiments are executed, it will export secondly,
which can also be skipped by setting `skip_output_write=True` for no files output.


In [5]:
multi_exps1 = experiment_randomized.multiOutput(
    config_list,
    summoner_name="qurrent.randomized_measure",  # you can name it whatever you want
    multiprocess_build=True,
    # Using multiprocessing to build the experiments,
    # it will be faster but take all the CPU
    skip_build_write=True,
    # Skip the writing of the experiment as files during the build,
    save_location=".",
    # Save the experiment as files in the current directory
    multiprocess_write=True,
    # Writing the experiment as files using multiprocessing,
)
multi_exps1

| MultiManager building...
| Write "qurrent.randomized_measure.001", at location "qurrent.randomized_measure.001"


| 180/180 100%|██████████| - Loading 180 experiments... - 00:33 < 00:00


| MultiOutput running...


| 180/180 100%|██████████| - Executing completed 'run.001', denoted date: 2025-05-22 14:39:57... - 01:14 < 00:00


| Export multimanager...


| 9/9 - Exporting done - 00:00 < 00:00                           

| No quantity to export.
| Export multi.config.json for d8146e57-30c9-44c9-a80b-dd761ac21252



| 179/179 - Exporting experiments... - 00:22 < 00:00
| 180/180 - Loading file infomation... - 00:00 < 00:00

| Exporting file taglist...
| Exporting qurrent.randomized_measure.001/qurryinfo.json...
| Exporting qurrent.randomized_measure.001/qurryinfo.json done.


'd8146e57-30c9-44c9-a80b-dd761ac21252'

In [6]:
experiment_randomized.multimanagers[multi_exps1]

<MultiManager(id="d8146e57-30c9-44c9-a80b-dd761ac21252",
  name="qurrent.randomized_measure.001",
  tags=(),
  jobstype="local",
  pending_strategy="tags",
  last_events={
    'output.001': '2025-05-22 14:39:58',},
  exps_num=180)>

### d. Run post-processing at once


In [7]:
experiment_randomized.multiAnalysis(
    summoner_id=multi_exps1,
    skip_write=True,
    multiprocess_write=False,
    selected_qubits=[0, 1],
)

| 180/180 - Analysis: Preparing error mitigation of selected qubits: [0, 1] - 00:08 < 00:00

| "report.001" has been completed.


'd8146e57-30c9-44c9-a80b-dd761ac21252'

In [8]:
print("| Available results:")
for k, v in (
    experiment_randomized.multimanagers[multi_exps1]
    .quantity_container["report.001"]
    .items()
):
    print("| -", k, "with length", len(v))

| Available results:
| - ('trivial_paramagnet', 'size_2') with length 10
| - ('ghz', 'size_2') with length 10
| - ('neel', 'size_2') with length 10
| - ('trivial_paramagnet', 'size_4') with length 10
| - ('ghz', 'size_4') with length 10
| - ('neel', 'size_4') with length 10
| - ('trivial_paramagnet', 'size_6') with length 10
| - ('ghz', 'size_6') with length 10
| - ('neel', 'size_6') with length 10
| - ('trivial_paramagnet', 'size_8') with length 10
| - ('ghz', 'size_8') with length 10
| - ('neel', 'size_8') with length 10
| - ('trivial_paramagnet', 'size_10') with length 10
| - ('ghz', 'size_10') with length 10
| - ('neel', 'size_10') with length 10
| - ('trivial_paramagnet', 'size_12') with length 10
| - ('ghz', 'size_12') with length 10
| - ('neel', 'size_12') with length 10


In [9]:
experiment_randomized.multimanagers[multi_exps1].quantity_container["report.001"][
    ("trivial_paramagnet", "size_10")
][:2]

[{'purity': np.float64(1.1030912399291992),
  'entropy': np.float64(-0.14155212542560228),
  'puritySD': np.float64(0.7874543179099595),
  'entropySD': np.float64(1.0298843815024916),
  'num_classical_registers': 10,
  'classical_registers': [0, 1],
  'classical_registers_actually': [0, 1],
  'all_system_source': 'independent',
  'purityAllSys': np.float64(2.598401222229004),
  'entropyAllSys': np.float64(-1.377624216246914),
  'puritySDAllSys': np.float64(4.032635600231429),
  'entropySDAllSys': np.float64(2.2390165661850303),
  'num_classical_registers_all_sys': 10,
  'classical_registers_all_sys': None,
  'classical_registers_actually_all_sys': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  'errorRate': np.float64(-0.612440289364807),
  'mitigatedPurity': np.float64(0.5781165980398923),
  'mitigatedEntropy': np.float64(0.79056760138383),
  'counts_num': 100,
  'taking_time': 0.000987924,
  'taking_time_all_sys': 0.033569723,
  'counts_used': None,
  'input': {'num_qubits': 10,
   'selected_qubit

### e. Run post-processing at once with specific analysis arguments

At first, we need to get the each experiment's id in the `multimanager` instance.


In [10]:
expkeys_of_multi_exps1 = list(
    experiment_randomized.multimanagers[multi_exps1].exps.keys()
)
print(len(expkeys_of_multi_exps1))

180


1. If you want to run the post-processing for some specific experiments,
   for example, the first 3 experiments we get for the `multimanager` instance.


In [11]:
experiment_randomized.multiAnalysis(
    summoner_id=multi_exps1,
    analysis_name="first_3",
    skip_write=True,
    multiprocess_write=False,
    specific_analysis_args={k: {
        "selected_qubits": [0, 1],   
    } if idx < 3 else False for idx, k in enumerate(expkeys_of_multi_exps1)},
)

| 180/180 - Analysis: Skipped a6f57199-f351-43c2-8502-73347659944a in d8146e57-30c9-44c9-a80b-dd761ac21252. - 00:00 < 00:00                     

| "first_3.002" has been completed.


'd8146e57-30c9-44c9-a80b-dd761ac21252'

In [12]:
print("| Available results:")
print(
    "| length:",
    sum(
        len(v)
        for v in experiment_randomized.multimanagers[multi_exps1]
        .quantity_container["first_3.002"]
        .values()
    ),
)

| Available results:
| length: 3


2. Or manually specify all the analysis arguments for each experiment.


In [13]:
experiment_randomized.multiAnalysis(
    summoner_id=multi_exps1,
    skip_write=False,
    analysis_name="all_manual",
    multiprocess_write=True,
    specific_analysis_args={k: {
        "selected_qubits": [0, 1],  # selected qubits for the analysis
    } for idx, k in enumerate(expkeys_of_multi_exps1)},
)

| 0/180 - Analysis: Preparing error mitigation of selected qubits: [0, 1] - 00:00 < ?                                                           

| 180/180 - Analysis: Preparing error mitigation of selected qubits: [0, 1] - 00:01 < 00:00                                                           


| "all_manual.003" has been completed.
| Export multimanager...


| 9/9 - Exporting done - 00:00 < 00:00                           
| 3/3 - exported quantity complete - 00:00 < 00:00    

| Export multi.config.json for d8146e57-30c9-44c9-a80b-dd761ac21252



| 179/179 - Exporting experiments... - 00:23 < 00:00
| 180/180 - Loading file infomation... - 00:00 < 00:00

| Exporting file taglist...
| Exporting qurrent.randomized_measure.001/qurryinfo.json...
| Exporting qurrent.randomized_measure.001/qurryinfo.json done.


'd8146e57-30c9-44c9-a80b-dd761ac21252'

In [14]:
print("| Available results:")
print(
    "| length:",
    sum(
        len(v)
        for v in experiment_randomized.multimanagers[multi_exps1]
        .quantity_container["all_manual.003"]
        .values()
    ),
)

| Available results:
| length: 180


### f. Read exported multimanager data


In [15]:
multi_exps1_reades = experiment_randomized.multiRead(
    save_location=".",
    summoner_name="qurrent.randomized_measure.001",
)

| Retrieve qurrent.randomized_measure.001...
| at: qurrent.randomized_measure.001


| 180/180 100%|██████████| - Loading 180 experiments ... - 00:07 < 00:00


In [16]:
expkeys_of_multi_exps1

['d7e5bde9-35d6-4480-846c-1ce135f12a22',
 '4108b5fd-de55-4671-a688-f36a8a8d7e7c',
 '11108337-10bd-437c-b61b-338e44f81c8e',
 'c4eabf32-d21f-4481-873f-7d81a6e7eb6c',
 'e78b4f39-d272-43d2-bfca-2b0590f047a1',
 '81c4a84d-c7a5-4091-b199-bf2415c03be2',
 '060c5100-2de1-46c4-9b32-8fcf3e3df3cc',
 '713153be-a43b-4e78-8183-7dafafb3259d',
 '57f39bbd-ee30-43ca-84e3-1fe7b97a5c6a',
 'b9e62941-da32-4cae-8dde-cb1fe132f7c0',
 '402f1713-6463-4f22-b804-0fd7318c1cff',
 '496d4543-8025-4e94-b6e2-a70254ab7588',
 'ed6f02de-ed34-4658-a0ca-969e29b8dd71',
 '957fee73-38ce-4947-beea-8b260224b296',
 '1cbdbe12-6cb5-4db5-badc-440939f5ebf8',
 '8e306ea1-1bf4-4bdf-a43a-64d9c3fb28ae',
 '7d059cc9-11e2-41b7-9555-8c31b695bab0',
 'd2e3e51a-37ec-4ee8-a308-fe91a80b33d5',
 'd973016c-a7f3-4227-a86a-241d99e2a766',
 'bee05788-253a-491f-be81-a3312c2a6db7',
 '8db85638-c3e0-4747-bb26-9f8b24a6f45e',
 'f9aef5b8-6712-4914-927b-6f8c1f187290',
 'f61b9335-553b-4d63-9ae3-d9be46e41a88',
 '6a72e672-ccfa-46d5-8a1b-8ef5264d7208',
 'ed3ffd03-d6da-

---

### Post-Process Availablities and Version Info

In [17]:
from qurry.process import AVAIBILITY_STATESHEET
AVAIBILITY_STATESHEET

 | Qurry version: 0.12.2.dev2
---------------------------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability ................... Python Cython Rust   JAX   
 - randomized_measure
   - entangled_entropy.entropy_core_2 ....... Yes    Depr.  Yes    No    
   - entangle_entropy.purity_cell_2 ......... Yes    Depr.  Yes    No    
   - entangled_entropy_v1.entropy_core ...... Yes    Depr.  Yes    No    
   - entangle_entropy_v1.purity_cell ........ Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_core_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_cell_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_core ...... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_cell ...... Yes    Depr.  Yes    No    
 - hadamard_test
   - purity_echo_core ....................... Yes    No     Yes    No    
 - magnet_square
   - magnsq_core ............................ Yes    No     No     No  